https://keras.io/preprocessing/image/

# TensorFlow 2.0

In [2]:
import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Hyperparameter Tunning

In [3]:
num_epochs = 10
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

## Preprocess

In [4]:
import pandas as pd

In [5]:
train_df = pd.read_csv('./train_dataset.csv')
test_df = pd.read_csv('./test_dataset.csv')

In [6]:
train_df.head()

,name,class
0,./dataset/cifar/train/23927_frog.png,frog
1,./dataset/cifar/train/3442_deer.png,deer
2,./dataset/cifar/train/11364_cat.png,cat
3,./dataset/cifar/train/17265_dog.png,dog
4,./dataset/cifar/train/36801_bird.png,bird


In [7]:
test_df.head()

,name,class
0,./dataset/cifar/test/2868_truck.png,truck
1,./dataset/cifar/test/7100_truck.png,truck
2,./dataset/cifar/test/6405_automobile.png,automobile
3,./dataset/cifar/test/5242_airplane.png,airplane
4,./dataset/cifar/test/2922_cat.png,cat


In [9]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0.3,
    zoom_range=0.2,
    horizontal_flip=True,
)

test_datagen = ImageDataGenerator(
    rescale=1./255
)

In [16]:
train_generator = train_datagen.flow_from_dataframe(
    train_df, 
    x_col='name', 
    y_col='class', 
    target_size=input_shape[:2],
    batch_size=batch_size
)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='name',
    y_col='class',
    target_size=input_shape[:2],
    batch_size=batch_size
)

Found 50000 validated image filenames belonging to 10 classes.
Found 10000 validated image filenames belonging to 10 classes.


## Build Model

In [17]:
inputs = layers.Input(input_shape)
net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [18]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

## Training

In [19]:
len(train_df) // batch_size

1562

In [20]:
len(train_generator)

1563

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=num_epochs,
    validation_data=test_generator,
    validation_steps=len(test_generator)
)

Epoch 1/10
1563/1563 [==============================] - 45s 29ms/step - loss: 1.7502 - accuracy: 0.3531 - val_loss: 1.4083 - val_accuracy: 0.4835
Epoch 2/10
1563/1563 [==============================] - 46s 30ms/step - loss: 1.4710 - accuracy: 0.4651 - val_loss: 1.3471 - val_accuracy: 0.5094
Epoch 3/10
1563/1563 [==============================] - 46s 30ms/step - loss: 1.3691 - accuracy: 0.5027 - val_loss: 1.1781 - val_accuracy: 0.5678
Epoch 4/10
1563/1563 [==============================] - 43s 28ms/step - loss: 1.3097 - accuracy: 0.5265 - val_loss: 1.0801 - val_accuracy: 0.6110
Epoch 5/10
1563/1563 [==============================] - 43s 27ms/step - loss: 1.2696 - accuracy: 0.5454 - val_loss: 1.1120 - val_accuracy: 0.5969
Epoch 6/10
1563/1563 [==============================] - 43s 27ms/step - loss: 1.2298 - accuracy: 0.5587 - val_loss: 1.2498 - val_accuracy: 0.5563
Epoch 7/10
1563/1563 [==============================] - 43s 27ms/step - loss: 1.2130 - accuracy: 0.5666 - val_loss: 1.0594 -